# [ParslFest 2025](https://parsl-project.org/parslfest/parslfest2025.html)

# [Accelerating QMCpy Notebook Tests with Parsl](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=1-37&t=WnKcu2QYO8JXvtpP-0)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QMCSoftware/QMCSoftware/blob/develop/demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb)

Joshua Herman, Brandon Sharp, and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

Updated: Dec 3, 2025


**Requirements**:

* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [2]:
import sys
import os
import parsl as pl

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

## 2. Parsl

1. Install and Configure Parsl
2. Run the tests in parallel with Parsl

### 2.1 Configure Parsl

In [3]:
from parsl.config import Config
from parsl.executors import ThreadPoolExecutor

# Prefer explicit PARSL_MAX_WORKERS from environment when provided by the caller
_env_workers = os.environ.get('PARSL_MAX_WORKERS')
if _env_workers:
    try:
        max_workers = int(_env_workers)
    except ValueError:
        max_workers = None
else:
    max_workers = None

# Default fallback based on CPU count (at least 1, cap to 16)
if not max_workers:
    cpu_count = os.cpu_count() or 2
    max_workers = min(4, max(1, cpu_count - 1))
# Clamp to reasonable bounds
max_workers = max(1, min(int(max_workers), max(1, (os.cpu_count() or 2) - 1), 16))

# Use ThreadPoolExecutor (works reliably on macOS and Linux)
config = Config(executors=[ThreadPoolExecutor(max_threads=max_workers, label="local_threads")])

# Ensure clean state: clear any existing Parsl config from previous runs
pl.clear()

# Now load the config
pl.load(config)
print(f"Parsl loaded with {max_workers} workers (PARSL_MAX_WORKERS env={os.environ.get('PARSL_MAX_WORKERS')})")

Parsl loaded with 8 workers (PARSL_MAX_WORKERS env=8)


### 2.2 Create a Parsl Test Runner

In [4]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")
print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

Functions:
- bash_app
- execute_parallel_tests
- generate_summary_report
- get_runtime
- main
- optimal_schedule
- print_schedule
- reload_parsl_config
- sort_by_runtime

Help for execute_parallel_tests:
Help on function execute_parallel_tests in module parsl_test_runner:

execute_parallel_tests()
    Execute all testbook tests in parallel using Parsl



In [5]:
# Verify Parsl configuration
print(f"Max workers configured: {max_workers}")
print(f"Active Parsl DFK: {pl.dfk()}")
print(f"Executors: {[executor.label for executor in pl.dfk().executors.values()]}")
if hasattr(config, 'executors'):
    for executor in config.executors:
        if hasattr(executor, 'max_workers_per_node'):
            print(f"Executor '{executor.label}' max_workers_per_node: {executor.max_workers_per_node}")

Max workers configured: 8
Active Parsl DFK: <parsl.dataflow.dflow.DataFlowKernel object at 0x10ef3a750>
Executors: ['local_threads', '_parsl_internal']


### 2.3 Run the Notebooks in Parallel with Parsl

In [6]:
import uuid
import subprocess
import re

execution_id = str(uuid.uuid4())[:8]
print(f"=== EXECUTION ID: {execution_id} ===")
print(f"Starting parallel test execution with {max_workers} workers...")

par_fname = os.path.join(output_dir, f"parallel_times_{max_workers}.csv")
par_output = os.path.join(output_dir, f"parallel_output_{max_workers}.txt")
is_linux = sys.platform.startswith("linux")

if (not os.path.exists(par_fname)) or force_compute:
    repo_root = _find_repo_root()
    
    if is_debug:
        tests = "tb_quickstart tb_qmcpy_intro tb_lattice_random_generator"
        cmd = ["make", "booktests_parallel_no_docker", f"TESTS={tests}"]
    else:
        cmd = ["make" if not is_linux else "make -j1", "booktests_parallel_no_docker"]
    if is_linux:
        cmd = ["taskset", "-c", "0"] + cmd
    
    # Propagate PARSL_MAX_WORKERS environment variable to subprocess
    env = os.environ.copy()
    env['PARSL_MAX_WORKERS'] = str(max_workers)
    
    with open(par_output, 'wb') as out_f:
        try:
            subprocess.run(cmd, cwd=repo_root, stdout=out_f, stderr=subprocess.STDOUT, check=True, env=env)
        except subprocess.CalledProcessError:
            pass
    
    # parse parallel time from output (sum of individual test times, not wall-clock)
    with open(par_output, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()
        match = re.search(r"Total test time: ([\d\.]+)s", text)
        if match:
            parallel_time = float(match.group(1))
        else:
            parallel_time = 0.0

    print(f"\n=== RESULTS FOR EXECUTION {execution_id} ===")
    print(f"Parallel time: {parallel_time:.2f} seconds")

    with open(par_fname, "w") as f:
        _ = f.write(f"workers,time\n")
        _ = f.write(f"{max_workers},{parallel_time:.2f}\n")
    
    print(f"=== END EXECUTION {execution_id} ===")

=== EXECUTION ID: 03c31d23 ===
Starting parallel test execution with 8 workers...


CompletedProcess(args=['make', 'booktests_parallel_no_docker'], returncode=0)


=== RESULTS FOR EXECUTION 03c31d23 ===
Parallel time: 524.10 seconds
=== END EXECUTION 03c31d23 ===


In [7]:
!date
!ls -ltr output

Wed Dec  3 19:52:49 CST 2025


total 240
-rw-r--r--@ 1 terrya  staff   6267 Dec  3 19:30 sequential_output.csv
-rw-r--r--@ 1 terrya  staff      7 Dec  3 19:30 sequential_time.csv
-rw-r--r--@ 1 terrya  staff   8549 Dec  3 19:30 01_sequential_output.ipynb
-rw-r--r--@ 1 terrya  staff   4590 Dec  3 19:39 parallel_output_1.txt
-rw-r--r--@ 1 terrya  staff     22 Dec  3 19:39 parallel_times_1.csv
-rw-r--r--@ 1 terrya  staff  13674 Dec  3 19:39 02_parallel_workers_1.ipynb
-rw-r--r--@ 1 terrya  staff   4592 Dec  3 19:44 parallel_output_2.txt
-rw-r--r--@ 1 terrya  staff     22 Dec  3 19:44 parallel_times_2.csv
-rw-r--r--@ 1 terrya  staff  13934 Dec  3 19:44 02_parallel_workers_2.ipynb
-rw-r--r--@ 1 terrya  staff   4592 Dec  3 19:49 parallel_output_4.txt
-rw-r--r--@ 1 terrya  staff     22 Dec  3 19:49 parallel_times_4.csv
-rw-r--r--@ 1 terrya  staff  14188 Dec  3 19:49 02_parallel_workers_4.ipynb
-rw-r--r--@ 1 terrya  staff   4592 Dec  3 19:52 parallel_output_8.txt
-rw-r--r--@ 1 terrya  staff     22 Dec  3 19:52 parallel_times

In [8]:
import platform

if platform.system().lower() == 'linux':
    !uname -a
    !nproc --all
    !awk '/MemTotal/ {printf "%.2f GB\n", $2/1024/1024}' /proc/meminfo